[← Algorithms as Dynamical Systems](../../getting_started/theory_to_python/algorithms_as_dynamical_systems.rst)

:::note
This notebook serves as a template for implementing state estimation algorithms as dynamical systems using pykal. Templating advice is boxed in blue.
:::

# Example: Kalman Filter

:::note
The title of the notebook should be the name of the algorithm. The text beneath the title should serve as a general overview of your algorithm, including links to useful materials. If your algorithm is included in the `pykal.algorithm_library`, then include a link to the API reference. We give an example below.
:::

The Kalman filter is a (linearly) optimal state estimation algorithm. The optimality is also conditional on the process and measurement noise being Gaussian. This algorithm works in two phases: a **prediction** phase and an **update** phase. For further information, read the [Wikipedia article](https://en.wikipedia.org/wiki/Kalman_filter). For Kalman's original paper on arxiv, which has a wonderful manifold projection interpretation of optimal estimation, read [the paper here ](https://www.cs.cmu.edu/~motionplanning/papers/sbp_papers/k/Kalman1960.pdf?utm_source=chatgpt.com).

The Kalman filter dynamical system as implemented in this notebook may be found in `pykal.algorithm_library.kf`.

The Kalman filter is an optimal state estimator for linear Gaussian systems. It recursively estimates the state of a dynamic system from noisy measurements by performing a predict-update cycle at each time step.

## Definition of Algorithm

:::note
Define the algorithm here. Derivation is unnecessary, but can be referenced using links. The definition must be self-contained; that is, all assumptions must be stated and variables must be defined. We demonstrate by example below.
:::

We model a discrete-time linear dynamical system with Gaussian noise by


$$
\begin{aligned}
x_{k+1} &= F_k x_k + B_k u_k + w_k, \\
y_k     &= H_k x_k + v_k.
\end{aligned}
$$

Where
- $x_k \in \mathbb{R}^n$ is the (hidden) state at time step $k$,
- $u_k \in \mathbb{R}^p$ is a known control input,
- $y_k \in \mathbb{R}^m$ is the measurement,
- $F_k \in \mathbb{R}^{n\times n}$ is the state transition matrix,
- $B_k \in \mathbb{R}^{n\times p}$ is the control-input matrix,
- $H_k \in \mathbb{R}^{m\times n}$ is the measurement matrix.

The process noise and measurement noise are modeled as zero-mean Gaussian random variables

$$
\begin{aligned}
w_k & \sim \mathcal{N}(0, Q_k)\\
v_k & \sim \mathcal{N}(0, R_k)
\end{aligned}
$$

with covariances $Q_k \in \mathbb{R}^{n\times n}$ and $R_k \in \mathbb{R}^{m\times m}$. We also assume an initial Gaussian prior

$$
x_0 \sim \mathcal{N}(\hat{x}_{0|0}, P_{0|0}).
$$

The Kalman filter produces, at each time $k$, a Gaussian posterior estimate of the state conditioned on measurements up to time $k$:

$$
p(x_k \mid y_{0:k}) \approx \mathcal{N}(\hat{x}_{k|k}, P_{k|k}),
$$

where $\hat{x}_{k|k}$ is the posterior mean (state estimate) and $P_{k|k}$ is the posterior covariance (uncertainty).

### Predict Step

Given $(\hat{x}_{k|k}, P_{k|k})$, the filter predicts the state at the next time step:

$$
\hat{x}_{k+1|k} = F_k \hat{x}_{k|k} + B_k u_k,
$$
$$
P_{k+1|k} = F_k P_{k|k} F_k^\top + Q_k.
$$

### Update Step

When a new measurement $y_{k+1}$ is observed, form the innovation (residual)

$$
r_{k+1} = y_{k+1} - H_{k+1}\hat{x}_{k+1|k},
$$
and its covariance

$$
S_{k+1} = H_{k+1} P_{k+1|k} H_{k+1}^\top + R_{k+1}.
$$

The Kalman gain is

$$
K_{k+1} = P_{k+1|k} H_{k+1}^\top S_{k+1}^{-1}.
$$

Then the posterior mean and covariance are updated by

$$
\hat{x}_{k+1|k+1} = \hat{x}_{k+1|k} + K_{k+1} r_{k+1},
$$
$$
P_{k+1|k+1} = (I - K_{k+1}H_{k+1}) P_{k+1|k}.
$$

A numerically safer equivalent covariance update (the Joseph form) is

$$
P_{k+1|k+1} = (I - K_{k+1}H_{k+1})P_{k+1|k}(I - K_{k+1}H_{k+1})^\top + K_{k+1} R_{k+1} K_{k+1}^\top.
$$

### Optimality Statement

Under the linear-Gaussian assumptions above, the Kalman filter computes the exact posterior mean and covariance, and $\hat{x}_{k|k}$ is the minimum mean-square error (MMSE) estimator of $x_k$ given $y_{0:k}$. Equivalently, among all estimators that are affine functions of the measurements, it is the best linear unbiased estimator (BLUE) when the relevant moments exist.

In practice, the Kalman filter is implemented as the recursive map

$$
(\hat{x}_{k|k}, P_{k|k}) \mapsto (\hat{x}_{k+1|k+1}, P_{k+1|k+1}),
$$

driven by inputs $(u_k, y_{k+1})$ and parameters $(F_k, B_k, H_{k+1}, Q_k, R_{k+1})$.

For a full derivation, see [Kalman's original paper](https://ntrs.nasa.gov/citations/19860016041) or [this tutorial](https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/).

## Algorithm as a Dynamical System
::: note
With reference to the definition of the algorithm as needed, define the dynamical system representation of the algorithm. As before, the definition must be self-contained; that is, all assumptions must be stated and variables must be defined. After the definition, show the evolution function (`f`) and output function (`h`) implementation in Python. We demonstrate by example below.
:::
To represent the Kalman Filter (KF) as a **discrete-time dynamical system**, we define the *algorithm state* to be the pair

$$
z_k := (\hat{x}_{k|k}, P_{k|k}),
$$

where $\hat{x}_{k|k} \in \mathbb{R}^n$ is the current state estimate (posterior mean) and $P_{k|k} \in \mathbb{R}^{n\times n}$ is the current posterior covariance. The algorithm is driven by a measurement input

$$
u_k := y_k \in \mathbb{R}^m,
$$

and (optionally) by an external control input $u_k^{\text{plant}}$ that is embedded inside the parameter dictionary passed to the model function.

We assume that the plant model and measurement model are available as **noise-free** functions

$$
f:\mathbb{R}^n \to \mathbb{R}^n, \qquad h:\mathbb{R}^n \to \mathbb{R}^m,
$$

together with matrices (or Jacobians) $F_k \in \mathbb{R}^{n\times n}$ and $H_k \in \mathbb{R}^{m\times n}$ used to propagate covariance and form the measurement update, and noise covariances $Q_k \succeq 0$ and $R_k \succ 0$.

With this setup, the Kalman filter is exactly the recursion

$$
z_{k+1} = \Phi(z_k, y_{k+1}; \theta_k),
$$

where $\theta_k$ denotes the collection of parameters required at time $k$ (e.g., the model functions, their parameters, and the matrices $F_k,H_k,Q_k,R_k$). Concretely, the state transition map $\Phi$ is the standard predict–update cycle:

**Predict**

$$
\hat{x}_{k+1|k} = f(\hat{x}_{k|k}; \theta_k),
$$
$$
P_{k+1|k} = F_k P_{k|k} F_k^\top + Q_k.
$$

**Update**

$$
\hat{y}_{k+1|k} = h(\hat{x}_{k+1|k}; \theta_k),
\qquad
r_{k+1} = y_{k+1} - \hat{y}_{k+1|k},
$$

$$
S_{k+1} = H_k P_{k+1|k} H_k^\top + R_k,
\qquad
K_{k+1} = P_{k+1|k} H_k^\top S_{k+1}^{-1},
$$

$$
\hat{x}_{k+1|k+1} = \hat{x}_{k+1|k} + K_{k+1} r_{k+1},
$$

and we update covariance using the Joseph form

$$
P_{k+1|k+1}
=
(I - K_{k+1}H_k)P_{k+1|k}(I - K_{k+1}H_k)^\top
+
K_{k+1}R_kK_{k+1}^\top,
$$

which is numerically stable and preserves positive semidefiniteness better in finite precision arithmetic.

Finally, we define the algorithm output as the estimated state:

$$
\psi(z_k) = \hat{x}_{k|k}.
$$

In `pykal`, this corresponds exactly to:
- **State transition**: `f(...)` implements $\Phi$ (one full predict–update step), taking $(\hat{x}_{k|k}, P_{k|k})$ and the new measurement $y_k$ (plus parameters) and returning $(\hat{x}_{k+1|k+1}, P_{k+1|k+1})$.
- **Output map**: `h(xhat_P)` implements $\psi$ by extracting $\hat{x}$ from the tuple $(\hat{x}, P)$.

We implement $\Phi$ as a callable with signature

$$
(\hat{x}_{k|k}, P_{k|k}) \mapsto (\hat{x}_{k+1|k+1}, P_{k+1|k+1})
$$

driven by measurement input $y_k$, with:
- `f(**f_params)` returning $\hat{x}_{k+1|k}$,
- `h(**h_params)` returning $\hat{y}_{k|k-1}$,
- `Fk, Hk` used for covariance propagation and update,
- `Qk, Rk` encoding process and measurement noise covariances.

In [ ]:

# In addition to importing the DynamicalSystem Module, import any other packages that are necessary here. For example, if your algorithm uses "scipy" or "cvxpy", import them below.


import typing  # for function signatures
from typing import Tuple, Callable, Dict
from numpy.typing import NDArray


def f(
    *,
    xhat_P: Tuple[NDArray, NDArray],
    yk: NDArray,
    f: Callable,
    f_params: Dict,
    h: Callable,
    h_params: Dict,
    Fk: NDArray,
    Qk: NDArray,
    Hk: NDArray,
    Rk: NDArray,
) -> Tuple[NDArray, NDArray]:
    """
    Perform one full **predict–update** step of the discrete-time Kalman Filter.

    Parameters
    ----------
    xhat_P : Tuple[NDArray, NDArray]
        A tuple ``(x_hat_k, P_k)`` containing:
            - ``x_hat_k`` : the current estimated state of the plant, shape (n,1)
            - ``P_k``     : the current state covariance, shape (n,n)

    yk : NDArray
        The measurement at time k, shape (m,1).

    f : Callable
        The plant evolution function used to propagate the estimated mean:
            ``x_pred = f(**f_params)``
        This should return the **noise-free** predicted state.

    f_params : Dict
        Dictionary of parameters passed to the evolution function ``f``.

    h : Callable
        The plant output function used to compute the predicted measurement:
            ``y_pred = h(**h_params)``
        This should return the **noise-free** predicted measurement.

    h_params : Dict
        Dictionary of parameters passed to the measurement function ``h``.

    Fk : NDArray
        The state-transition Jacobian evaluated at the current estimate,
        used to propagate the covariance. Shape (n,n).

    Qk : NDArray
        The process-noise covariance matrix at time k, shape (n,n).

    Hk : NDArray
        The measurement Jacobian evaluated at the current estimate,
        used in the update. Shape (m,n).

    Rk : NDArray
        The measurement-noise covariance matrix at time k, shape (m,m).


    Returns
    -------
    (x_upd, P_upd) : Tuple[NDArray, NDArray]
        The updated state estimate and covariance:
            - ``x_upd`` : updated state estimate, shape (n,1)
            - ``P_upd`` : updated state covariance, shape (n,n)


    Notes
    -----
    This implementation follows the standard **linearized EKF equations**:

    **Predict step**
    ----------------
    State prediction:
        ``x_pred = f(**f_params)``

    Covariance prediction:
        ``P_pred = Fk @ Pk @ Fk.T + Qk``

    **Update step**
    ---------------
    Innovation:
        ``innovation = yk - y_pred``
        where ``y_pred = h(**h_params)``

    Innovation covariance:
        ``Sk = Hk @ P_pred @ Hk.T + Rk``

    Kalman gain:
        ``Kk = P_pred @ Hk.T @ Sk^{-1}``

    State update:
        ``x_upd = x_pred + Kk @ innovation``

    Covariance update (Joseph form for numerical stability):
        ``P_upd = (I - Kk @ Hk) @ P_pred @ (I - Kk @ Hk).T + Kk @ Rk @ Kk.T``

    The covariance matrix is explicitly symmetrized at the end to counter
    numerical drift:
        ``P_upd = 0.5 * (P_upd + P_upd.T)``
    """

    # === Extract covariance ===
    _, Pk = xhat_P

    # === Predict ===
    x_pred = f(**f_params)
    P_pred = Fk @ Pk @ Fk.T + Qk

    # === Innovation ===
    y_pred = h(**h_params)
    innovation = yk - y_pred

    # === Update ===
    Sk = Hk @ P_pred @ Hk.T + Rk
    ridge = 1e-9 * np.eye(Sk.shape[0])
    try:
        Sk_inv = np.linalg.inv(Sk + ridge)
    except np.linalg.LinAlgError:
        Sk_inv = np.linalg.pinv(Sk + ridge)

    Kk = P_pred @ Hk.T @ Sk_inv
    x_upd = x_pred + Kk @ innovation

    I = np.eye(P_pred.shape[0])
    P_upd = (I - Kk @ Hk) @ P_pred @ (I - Kk @ Hk).T + Kk @ Rk @ Kk.T
    P_upd = 0.5 * (P_upd + P_upd.T)

    return (x_upd, P_upd)


def h(xhat_P: Tuple[NDArray, NDArray]) -> NDArray:
    return xhat_P[0]  # extracts current state estimate

## Example: 1D Constant Velocity Tracking
::: note
Finally, show an example of your algorithm in action.
:::
We track a target moving in **1D with (approximately) constant velocity**, where we only observe its **position**. We also include process and measurement noise using `pykal.data_change.corrupt`.

We consider the 1D constant-velocity model with state

$$
x_k = \begin{bmatrix} p_k \\ v_k \end{bmatrix},
$$

where $p_k$ is position and $v_k$ is velocity at discrete time $k$.

The discrete-time dynamics with sampling interval $\Delta t$ are:

$$
x_{k+1} =
\begin{bmatrix}
1 & \Delta t \\
0 & 1
\end{bmatrix}
x_k + w_k,
$$

where $w_k \sim \mathcal{N}(0, Q)$ is process noise (acceleration noise).

The measurement model observes **position only**:

$$
y_k =
\begin{bmatrix}
1 & 0
\end{bmatrix}
x_k + v_k,
$$

where $v_k \sim \mathcal{N}(0, R)$ is measurement noise. If we model unknown accelerations as white noise, a standard discrete-time covariance is:

$$
Q = \sigma_a^2
\begin{bmatrix}
\frac{\Delta t^4}{4} & \frac{\Delta t^3}{2} \\
\frac{\Delta t^3}{2} & \Delta t^2
\end{bmatrix},
$$

and the measurement covariance is:

$$
R = \sigma_y^2.
$$

In this example we choose:

- process noise covariance: $Q = \begin{bmatrix} 10^{-4} & 0 \\ 0 & 10^{-4} \end{bmatrix}$
- measurement noise covariance: $R = \begin{bmatrix} 10^{-3} \end{bmatrix}$

In [ ]:
from pykal import DynamicalSystem
import matplotlib.pyplot as plt  # for plotting


def target_f(xk, Ak):
    """Noise-free state evolution."""
    xk_next = Ak @ xk
    return xk_next


def target_h(xk, Ck):
    """Noise-free measurement."""
    yk = Ck @ xk
    return yk


target = DynamicalSystem(f=target_f, h=target_h, state_name="pos_vel")

kf = DynamicalSystem(f=f, h=h, state_name="xest_P")  # define the kf dynamical system

In [ ]:
import numpy as np
import pandas as pd
from pykal.data_change import corrupt

# Define covariances
Qk = np.array([[1e-4, 0], [0, 1e-4]])
Rk = np.array([[1e-3]])
Pk = np.array([[1e-3, 0], [0, 1e-3]])

# Initial state (column vector)
xk = np.array([[0.0], [1.0]])
xk_kf = [xk, Pk]
measurements = []
kf_state_estimates = []

dt = 0.1
for tk in np.arange(0, 10, dt):

    Ak = np.array([[1, dt], [0, 1]])
    Ck = np.array([[1, 0]])

    # Noise-free state evolution
    xk = target.f(xk=xk, Ak=Ak)

    # Apply process noise using pykal.data_change.corrupt
    xk = corrupt.with_gaussian_noise(xk.flatten(), Q=Qk).reshape(-1, 1)

    # Noise-free measurement
    yk = target.h(xk=xk, Ck=Ck)

    # Apply measurement noise using pykal.data_change.corrupt
    yk = corrupt.with_gaussian_noise(yk.flatten(), Q=Rk).reshape(-1, 1)

    xk_kf, yk_kf = kf.step(
        return_state=True,
        param_dict={
            "xhat_P": xk_kf,
            "yk": yk,
            "f": target_f,
            "f_params": {"xk": xk_kf[0], "Ak": Ak},
            "h": target_h,
            "h_params": {"xk": xk_kf[0], "Ck": Ck},
            "Fk": Ak,  # linear system means Fk and Hk are just Ak and Ck
            "Hk": Ck,
            "Qk": Qk,
            "Rk": Rk,
        },
    )

    measurements.append(yk)
    kf_state_estimates.append(yk_kf)

meas_df = pd.DataFrame(np.vstack(measurements), columns=["Position"])
meas_df.plot()

kf_state_est_df = pd.DataFrame(
    np.array(kf_state_estimates).squeeze(),
    columns=["Estimated Position", "Estimated Velocity"],
)
kf_state_est_df.plot()


## Notes on Usage
Add notes you think users of this algorithm should know.

This section provides practical guidance on when and how to use the Kalman filter implementation.

### When to Use This Algorithm

The Kalman filter is appropriate when **all** of the following conditions hold:

1. **Linear dynamics**: The system evolution and measurement models are linear (or have been linearized)
2. **Gaussian noise**: Process noise and measurement noise are approximately Gaussian
3. **Known covariances**: The noise covariance matrices $Q$ and $R$ can be estimated or characterized
4. **Real-time requirements**: You need recursive, online state estimation with constant memory

If your system is **nonlinear**, consider using the Extended Kalman Filter (EKF) or Unscented Kalman Filter (UKF) instead. If your noise is **non-Gaussian** (e.g., heavy-tailed, multimodal), consider particle filters or other robust estimators.

### Common Applications

The Kalman filter is widely used in:

- **Navigation and tracking**: GPS/IMU fusion, target tracking, vehicle localization
- **Robotics**: State estimation for mobile robots, quadrotors, manipulators
- **Control systems**: Observer design for output feedback control
- **Signal processing**: Noise reduction, sensor fusion, time series prediction
- **Economics**: Estimating hidden states in econometric models

### Jacobian Requirements

For **linear systems**, the Jacobians are simply the system matrices:

- State transition Jacobian: $F_k$ is the linearized dynamics matrix
- Measurement Jacobian: $H_k$ is the linearized measurement matrix

In this implementation:
- `Fk` should be the Jacobian $\frac{\partial f}{\partial x}$ evaluated at the current estimate
- `Hk` should be the Jacobian $\frac{\partial h}{\partial x}$ evaluated at the current estimate

For **time-invariant linear systems** (constant matrices), these are the same at every time step. For **time-varying or nonlinear systems**, recompute the Jacobians at each step.

### Implementation Details

**State representation**: The algorithm state is a tuple `(xhat, P)` where:
- `xhat`: state estimate as a column vector, shape `(n, 1)`
- `P`: covariance matrix, shape `(n, n)`

**Plant model functions**: Provide noise-free functions `f` and `h`:
- `f(**f_params)`: returns the predicted state (no process noise)
- `h(**h_params)`: returns the predicted measurement (no measurement noise)

**Noise handling**: Apply noise corruption separately using `pykal.data_change.corrupt` during simulation, as shown in the example. This separates the deterministic model from stochastic effects.

**Numerical stability**: This implementation uses:
- Joseph form covariance update for numerical stability
- Ridge regularization (`1e-9 * I`) to avoid singular matrices
- Pseudo-inverse fallback if matrix inversion fails
- Explicit symmetrization of covariance to counter numerical drift

### Tuning Guidance

The performance of the Kalman filter depends critically on the noise covariance matrices:

- **Process noise $Q$**: Larger values → filter trusts model less, adapts faster to changes
- **Measurement noise $R$**: Larger values → filter trusts measurements less, smoother estimates

**Initial covariance $P_0$**: Set based on initial uncertainty. Large values indicate high initial uncertainty; the filter will converge as measurements arrive.

For practical tuning, see [this guide on tuning Kalman filters](https://shepherdmoon.org/kalman_filter/tuning.html).

[← Algorithms as Dynamical Systems](../../getting_started/theory_to_python/algorithms_as_dynamical_systems.rst)